In [1]:
import re
from transformers import pipeline
import torch

/Users/dominik/Desktop/University/Semestr5/ModeleJezykowe/.env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
polka = pipeline("text-generation", model="eryk-mazus/polka-1.1b")
model = polka.model 
tokenizer = polka.tokenizer

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [3]:
def is_math_question(question):
    math_keywords = ['ile to jest', 'oblicz', 'jaki jest wynik', 'policz', 'wykonaj działanie']
    return any(keyword in question.lower() for keyword in math_keywords)

def answer_math_question(question):
    try:
        patterns = [
            r'ile to jest (.*)\?',
            r'oblicz (.*)\.',
            r'jaki jest wynik (.*)\?',
            r'policz (.*)\.',
            r'wykonaj działanie (.*)\.'
        ]
        expression = None
        for pattern in patterns:
            match = re.findall(pattern, question.lower())
            if match:
                expression = match[0]
                break
        if expression is None:
            return "Nie potrafię znaleźć wyrażenia do obliczenia."
        # Zamiana słów na operatory
        expression = expression.replace('plus', '+').replace('minus', '-').replace('razy', '*')\
                               .replace('podzielić przez', '/').replace('podzielone przez', '/')\
                               .replace(' ', '')
        # Bezpieczne obliczenie wyrażenia
        result = eval(expression)
        return str(result)
    except Exception:
        return "Nie potrafię obliczyć tego wyrażenia."

In [4]:
def is_translation_question(question):
    translation_keywords = ['jak po polsku jest', 'co oznacza', 'przetłumacz', 'jak przetłumaczyć']
    return any(keyword in question.lower() for keyword in translation_keywords)

def answer_translation_question(question):
    patterns = [
        r'co oznacza (.*) po polsku\?',
        r'jak po polsku jest (.*)\?',
        r'przetłumacz (.*)\.',
        r'jak przetłumaczyć (.*) na polski\?'
    ]
    word = None
    for pattern in patterns:
        match = re.findall(pattern, question.lower())
        if match:
            word = match[0].strip().strip("'\"")
            break
    if word:
        possible_translations = ['kot', 'pies', 'dom', 'samochód', 'drzewo']
        best_translation = ''
        best_score = float('-inf')
        for translation in possible_translations:
            input_text = f"Słowo '{word}' oznacza po polsku '{translation}'."
            inputs = tokenizer.encode(input_text, return_tensors='pt')
            with torch.no_grad():
                outputs = model(inputs)
                log_probs = torch.nn.functional.log_softmax(outputs.logits, dim=-1)
                log_probs = log_probs[:, :-1, :]
                target_tokens = inputs[:, 1:]
                log_probs_target = log_probs.gather(2, target_tokens.unsqueeze(-1)).squeeze(-1)
                score = log_probs_target.sum().item()
                if score > best_score:
                    best_score = score
                    best_translation = translation
        return best_translation if best_translation else "Nie znam tłumaczenia."
    else:
        return "Nie potrafię przetłumaczyć tego słowa."


In [9]:
def generate_answer(question):
    if is_math_question(question):
        return answer_math_question(question)
    elif is_translation_question(question):
        return answer_translation_question(question)
    else:
        inputs = tokenizer.encode(question + " Odpowiedź:", return_tensors='pt')
        outputs = model.generate(inputs, max_new_tokens=80, num_return_sequences=1)
        answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

        return answer.replace(question + " Odpowiedź:", "").strip()

In [9]:
questions = [
    "Ile to jest 7 razy 8?",
    "Oblicz 15 plus 27.",
    "Jaki jest wynik 9 podzielone przez 3?",
    "Policz 12 minus 5.",
    "Wykonaj działanie 6 razy 7.",
    "Jak po polsku jest 'cat'?",
    "Co oznacza 'dog' po polsku?",
    "Przetłumacz 'house'.",
    "Jak przetłumaczyć 'tree' na polski?",
    "Jaka jest stolica Francji?",
    "Kto jest prezydentem Polski?"
]

for question in questions:
    answer = generate_answer(question)
    print("Pytanie:", question)
    print("Odpowiedź:", answer)
    print()

Pytanie: Ile to jest 7 razy 8?
Odpowiedź: 56

Pytanie: Oblicz 15 plus 27.
Odpowiedź: 42

Pytanie: Jaki jest wynik 9 podzielone przez 3?
Odpowiedź: 3.0

Pytanie: Policz 12 minus 5.
Odpowiedź: 7

Pytanie: Wykonaj działanie 6 razy 7.
Odpowiedź: 42



Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Pytanie: Jak po polsku jest 'cat'?
Odpowiedź: kot

Pytanie: Co oznacza 'dog' po polsku?
Odpowiedź: pies

Pytanie: Przetłumacz 'house'.
Odpowiedź: dom



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Pytanie: Jak przetłumaczyć 'tree' na polski?
Odpowiedź: drzewo



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Pytanie: Jaka jest stolica Francji?
Odpowiedź: Paryż! | Blog podróżniczy\n Paryż!\nFrancja to kraj, który koj

Pytanie: Kto jest prezydentem Polski?
Odpowiedź: Andrzej Duda - WP Wiadomości\n Andrzej Duda\nAndrzej Duda został wybrany na prezy



In [10]:
def evaluate_answers(question_file, answer_file, limit = None):
    with open(question_file, 'r', encoding='utf-8') as qf, open(answer_file, 'r', encoding='utf-8') as af:
        questions = qf.readlines()
        answers = af.readlines()

    correct = 0
    total = 0
    
    for question, expected_answer in zip(questions, answers):
        if limit is not None and total >= limit:
            break
        total += 1
        question = question.strip()
        expected_answer = expected_answer.strip()
        
        generated_answer = generate_answer(question).strip()

        if generated_answer.lower() == expected_answer.lower():
            correct += 1

        print(f"Pytanie: {question}")
        print(f"Oczekiwana odpowiedź: {expected_answer}")
        print(f"Wygenerowana odpowiedź: {generated_answer}")
        print(f"{'Poprawna' if generated_answer.lower() == expected_answer.lower() else 'Niepoprawna'}\n")
    
    accuracy = correct / total * 100 if total > 0 else 0
    print(f"Accuracy: {accuracy:.2f}%")
    return accuracy

evaluate_answers("task4_questions.txt", "task4_answers.txt", 10)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Pytanie: Która organizacja powstała wcześniej: Europejska Wspólnota Węgla i Stali czy Europejska Wspólnota Energii Atomowej?
Oczekiwana odpowiedź: Europejska Wspólnota Węgla i Stali
Wygenerowana odpowiedź: Europejska Wspólnota Energii Atomowej, która powstała w 1957 r. w wyniku połączenia Europejskiej Wspólnoty Węgla i Stali oraz Europejskiej Wspólnoty Energii Atomowej. Europejska Wspólnota Energii Atomowej została powołana w celu st
Niepoprawna



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Pytanie: W którym wieku został odlany dzwon Zygmunta?
Oczekiwana odpowiedź: w XVI	16
Wygenerowana odpowiedź: 100 lat temu - Dziennik Wschodni\n 100 lat temu\n100 lat temu odlano pierwszy dzwon Zygmunta w Lublinie. (fot. archiwum)\n100 lat temu odl
Niepoprawna



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Pytanie: Jak nazywał się gigantyczny goryl, bohater filmów japońskich?
Oczekiwana odpowiedź: King Kong
Wygenerowana odpowiedź: „Kamikaze” | naTemat.pl\n „Kamikaze” •\n „Kamikaze”\nJak nazywał się gig
Niepoprawna



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Pytanie: Jak nazywa się pojedynczy element schodów oraz cyfrowa ocena postępów ucznia?
Oczekiwana odpowiedź: stopień
Wygenerowana odpowiedź: to jest to, co nazywamy schodami. Schody są jednym z najbardziej popularnych przedmiotów do nauki, ponieważ są one bardzo łatwe do nauczenia się i są bardzo łatwe do nauczenia się. Ponadto, schody są bardzo łatwe do nauczenia się i są bardzo
Niepoprawna

Pytanie: Który kolumbijski pisarz urodzony w 1927 roku jest autorem powieści „Sto lat samotności”?
Oczekiwana odpowiedź: Gabriel García Márquez	Márquez
Wygenerowana odpowiedź: „Papierowy anioł”.\n1. „Papierowy anioł” to powieść o tym, jak w czasach wojny domowej w Kolumbii, 1948 roku, młody chłopak, Juan Carlos, trafia do więzienia i tam poznaje tajemniczą dziewczynę, która naz
Niepoprawna

Accuracy: 0.00%


0.0